In [ ]:
%load_ext nb_black

Welcome to this lab session 2 on Time series modeling for air pollution monitoring with a focus on the
calibration of low-cost sensors.

This lab session is based on the data and methods provided in the study by [Ellen M. Considine et al](https://www.sciencedirect.com/science/article/pii/S0269749120365222).

In this notebook we perform exploratory data analysis on our cleaned data from session 1.

In statistics, exploratory data analysis is an approach of analyzing data sets to summarize their main characteristics, often using statistical graphics and other data visualization methods. - [Wikipedia](https://en.wikipedia.org/wiki/Exploratory_data_analysis)

We will be asking ourselves the following questions:
    
- What is the geospatial context of the locations of our sensors (and the data they collect)?
- What is the length of arterial roads with 500 meter radius from each of the monitor locations?
- What are the summary statistics of the airnow and CS PM2.5, and what can we observe from these statistics?
- Using scatter plots and histograms to deepen our understanding of the possible disparities between airnow and CS readings. Here, we want to observe if there is a linear relationship between these readings.
- What happens on weekends? Is there a significant difference?
- Correlation matrix and scatter matrix between variables

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

import geopandas
import contextily as cx

# Load the training data from disk

The data is stored in a `.csv`file (with column names as keys to access the different time series variables). Pandas dataframes give use the possibility to manilpulate these kinds of files

In [ ]:
data_root = "./data/"
training_data_path = data_root + "cleaned_training.csv"
test_data_path = data_root + "cleaned_test.csv"

training_data = pd.read_csv(training_data_path)
test_data = pd.read_csv(test_data_path)

full_data = pd.concat([training_data, test_data], axis=0)

In [ ]:
full_data.dtypes

In [ ]:
full_data.head()

Any NaN values?

In [ ]:
full_data.isna().sum()

In [ ]:
full_data["date_time"] = pd.to_datetime(full_data["date_time"])

Now, we want to split our dataframe by sensor because we will be using these different sections of our data multiple times in this notebook.

# 1. Plot latitude and langitude of each sensing location with basemap to provide context.

Here we will use a `groupby` function in pandas. It is used to group large amounts of data by descrete values contained in columns of the data.

The `pandas.groupby.nth()` function is used to get the value corresponding the nth row for each group. To get the first value in a group, pass 0 as an argument to the `nth()` method. 

In [ ]:
locations = (
    full_data[["airnow_sensor", "longitude", "latitude"]]
    .groupby(by="airnow_sensor")
    .nth(0)
    .reset_index()
)

In [ ]:
locations

The `contextily` package is used to retrieve web map tiles from a number of sources (OpenStreetMap, Stamen). We can use this package to add basemap to enhance `geopandas.GeoDataFrame`plots.

In [ ]:
geo_df = geopandas.GeoDataFrame(
    locations,
    geometry=geopandas.points_from_xy(locations.longitude, locations.latitude),
)

geo_df = geo_df.set_crs("epsg:4326")

In [ ]:
geo_df

In [ ]:
geo_df.dtypes

In [ ]:
ax = geo_df.plot(
    figsize=(10, 10),
    alpha=1,
    edgecolor="b",
    legend=True,
    markersize=500,
    legend_kwds={"labels": geo_df["airnow_sensor"]},
)

for x, y, label in zip(geo_df.geometry.x, geo_df.geometry.y, geo_df["airnow_sensor"]):
    ax.annotate(
        label,
        xy=(x, y),
        xytext=(3, 3),
        textcoords="offset points",
        fontsize=25,
        color="r",
    )
plt.title("Locations of the AirNow (FEM) sensors used as reference in this study")
cx.add_basemap(ax, crs=geo_df.crs)

- The National Jewish Hospital (NJH) is farther away from the highway than the rest of the monitor locations. CAMP is also reltively far from large roads. This should have potential impact on how much and the variability of the PM2.5 atmospheric content sensed in these locations.

# 2. Exploring road length variables

Here we want to explore the lengths of arterial (Large) roads within 500m buffer surrounding each monitor location. We do this using a bar plot/chart.

A bar chart or bar graph is a chart or graph that presents categorical data with rectangular bars with heights or lengths proportional to the values that they represent. - [Wikipedia](https://en.wikipedia.org/wiki/Bar_chart)

In [ ]:
full_data_roads = (
    full_data.groupby(by="airnow_sensor")
    .nth(0)
    .reset_index()[["airnow_sensor", "a_road_500"]]
)

In [ ]:
full_data_roads

### Exercise 1: Obtain a bar plot of the road length for each monitor location using the given dataframe

Can you observe for 

**Follow up: What can you observe for from the plot?**

**Follow up: What can you observe (differently) for training and test set from the plot?**

Training: NJH, i25_glo, la_casa <br>
Test: CAMP, i25_denver

**Follow up: If you observed anything, how can we verify what you have observed?**

In [ ]:
plt.bar(full_data_roads["airnow_sensor"], full_data_roads["a_road_500"])
plt.show()

# 3. Summary statistics of our data

First, we obtain the summary statistics our "true" data, i.e the data from our monitor sensors (named `pm_airnow`) in our dataframe.

###### Training

In [ ]:
training_data.groupby("cs_sensor")[["pm_airnow"]].describe()

###### Test

In [ ]:
test_data.groupby("cs_sensor")[["pm_airnow"]].describe()

**Observations**

- NJH in training and CAMP in test data show relatively lower median values<br>
- The IQR for both NJH and CAMP are also lower than the rest.


### Summary statistics of the CS (low-cost) sensor data?

Remember, the column named `cs_sensor`provides the CS sensor tag for each data row and column named `pm_cs` contains the CA sensor data

###### Training

In [ ]:
training_data.groupby("cs_sensor")[["pm_cs"]].describe()

Box plots show the median, quartiles ($Q_1$, $Q_2$, $Q_3$, $Q_4$), interquartile range (IQR), minimum, maximum, and outlier points.

Low outlier points are defined as points below: $Q_1 − (1.5* IQR)$ <br>
High outlier points are defined as points above: $Q_3 + (1.5* IQR)$

$50\%$ of the lies between $Q_1$ and $Q_3$.

In [ ]:
training_data[["cs_sensor", "pm_cs"]].groupby("cs_sensor")[["pm_cs"]].boxplot(
    subplots=False, figsize=(10, 7)
)
plt.show()

###### Test

In [ ]:
test_data.groupby("cs_sensor")[["pm_cs"]].describe()

In [ ]:
test_data[["cs_sensor", "pm_cs"]].groupby("cs_sensor")[["pm_cs"]].boxplot(
    subplots=False, figsize=(10, 7)
)
plt.show()

- Take note of the IQR for CAMP and NJH. The IQR in these locations are lower than in the other locations. CAMP and NJH are farther away from the express way (see map).

# 4.  Scatterplot comparing AirNow and CS PM2.5 

A scatterplot shows the relationship between two quantitative variables measure in the same unit/space

In [ ]:
plt.scatter(full_data["pm_airnow"], full_data["pm_cs"])
plt.ylabel("PM2.5 - CS)", size=15)
plt.xlabel("PM2.5 - AirNow", size=15)
plt.title("Comparing AirNow  and CS sensor \n readings", size=15)
plt.axis("square")
plt.show()

In [ ]:
ax = sns.regplot(
    x=full_data["pm_airnow"],
    y=full_data["pm_cs"],
    color="green",
    line_kws={"color": "black"},
)

ax.set(xlabel="PM2.5 - CS", ylabel="PM2.5 - AirNow")
plt.axis("square")
plt.show()

**What can we do with this plot?** <br>
We can understand the relationship between our "true" PM2.5 values and the ones mesaured from our low-cost sensors


- The need for correction. A linear model is useful for this.
- A linear model could find the line of best fit here. We might not even need data other confounding variables to have a good starting model.

We can split our data by CS sensors to observe if there are any significant changes with respect to what has been observed over the whole data.

In [ ]:
njh_data = full_data[full_data["cs_sensor"] == "NJH"]
i25_glo1_data = full_data[full_data["cs_sensor"] == "i25_glo_1"]
i25_glo2_data = full_data[full_data["cs_sensor"] == "i25_glo_2"]
i25_glo3_data = full_data[full_data["cs_sensor"] == "i25_glo_3"]
lacasa_data = full_data[full_data["cs_sensor"] == "la_casa"]
camp_data = full_data[full_data["cs_sensor"] == "CAMP"]
i25_denver_data = full_data[full_data["cs_sensor"] == "i25_denver"]

In [ ]:
fig, [(ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)] = plt.subplots(
    nrows=2, ncols=4, figsize=(20, 5), sharex=True, sharey=True
)

ax1.scatter(njh_data["pm_airnow"], njh_data["pm_cs"])
ax1.set_title("NJH")

ax2.scatter(i25_glo1_data["pm_airnow"], i25_glo1_data["pm_cs"])
ax2.set_title("i-25 Globeville 1")
ax3.scatter(i25_glo2_data["pm_airnow"], i25_glo2_data["pm_cs"])
ax3.set_title("i-25 Globeville 2")
ax4.scatter(i25_glo3_data["pm_airnow"], i25_glo3_data["pm_cs"])
ax4.set_title("i-25 Globeville 3")
ax5.scatter(lacasa_data["pm_airnow"], lacasa_data["pm_cs"])
ax5.set_title("La Casa")
ax6.scatter(camp_data["pm_airnow"], camp_data["pm_cs"])
ax6.set_title("CAMP")

ax7.scatter(i25_denver_data["pm_airnow"], i25_denver_data["pm_cs"])
ax7.set_title("Denver")

ax8.axis("off")

ax1.set_ylabel("Monitor sensors", size=15)
fig.text(0.5, 0.04, "Low cost (CS) sensors", ha="center", va="center", size=15)

plt.show()

What do we observe? Any differences across the different CS sensors

- Some potential outlier measures at I-25 Globeville 1

# 4.  Historgrams comparing AirNow and CS PM2.5 

What histograms can tell us about the ditribution of our data:

- Is it unimodal, bimodal, or multimodal?
- how widely is the distribution spread?
- Do the distributions overlap for the different sensors?

###### Choosing bin size.

One simple rule is [Sturge’s rule](https://www.researchgate.net/publication/230257056_Sturges'_rule)

$K = 1 + 3.22 log(N)$

where,

K is the number of bins

N is the number of observations

In [ ]:
def bin_size(n_observations):

    return math.ceil(1 + (3.22 * math.log(n_observations)))

In [ ]:
plt.hist(
    [full_data["pm_airnow"], full_data["pm_cs"]],
    bins=bin_size(len(full_data)),
)
plt.xlabel("Value", size=15)
plt.ylabel("Frequency", size=15)
plt.legend()
plt.show()

In [ ]:
labels = ["AirNow", "CS"]
fig, [(ax1, ax2, ax3, ax4), (ax5, ax6, ax7, ax8)] = plt.subplots(
    nrows=2, ncols=4, figsize=(20, 9)
)  # , sharex=True, sharey=True)

ax1.hist(
    [njh_data["pm_airnow"], njh_data["pm_cs"]],
    bins=bin_size(len(njh_data)),
)
ax1.set_title("NJH")


ax2.hist(
    [i25_glo1_data["pm_airnow"], i25_glo1_data["pm_cs"]],
    bins=bin_size(len(i25_glo1_data)),
)
ax2.set_title("i-25 Globeville 1")

ax3.hist(
    [i25_glo2_data["pm_airnow"], i25_glo2_data["pm_cs"]],
    bins=bin_size(len(i25_glo2_data)),
)
ax3.set_title("i-25 Globeville 2")

ax4.hist(
    [i25_glo3_data["pm_airnow"], i25_glo3_data["pm_cs"]],
    bins=bin_size(len(i25_glo3_data)),
)
ax4.set_title("i-25 Globeville 3")

ax5.hist(
    [lacasa_data["pm_airnow"], lacasa_data["pm_cs"]],
    bins=bin_size(len(lacasa_data)),
)
ax5.set_title("La Casa")


ax6.hist(
    [camp_data["pm_airnow"], camp_data["pm_cs"]],
    bins=bin_size(len(lacasa_data)),
)
ax6.set_title("CAMP")

ax7.hist(
    [i25_denver_data["pm_airnow"], i25_denver_data["pm_cs"]],
    bins=bin_size(len(i25_denver_data)),
)
ax7.set_title("I-25 Denver")

ax8.axis("off")

ax1.set_ylabel("Probability", size=15)
fig.text(0.5, 0.04, "Measured PM2.5 value", ha="center", va="center", size=15)
fig.legend(
    labels,
    loc="lower right",
    bbox_to_anchor=(0.5, -0.04),
    ncol=len(labels),
    bbox_transform=fig.transFigure,
)
plt.show()

# What happens on weekends?

In [ ]:
plt.plot(camp_data["pm_airnow"], label="AirNow")
plt.plot(camp_data["pm_cs"], label="CS")
plt.bar(
    x=range(0, len(camp_data)),
    height=[x * 30 for x in camp_data["weekend"]],
    alpha=0.2,
    label="weekend",
)

plt.title("CAMP data - CS, Airnow and Bar plot show weekend days")
plt.legend()
plt.show()

In [ ]:
weekend_median = full_data[full_data["weekend"] == 1]["pm_airnow"].median()
weekday_median = full_data[full_data["weekend"] != 1]["pm_airnow"].median()

In [ ]:
plt.bar(x=["week day", "weekend"], height=[weekday_median, weekend_median])
plt.show()

There is lower PM2.5 on weekends than on weekdays. This could be due to reduction in industrial activities, traffic on highways and other sources of pollution over the weekend

In [ ]:
corr = full_data[["pm_airnow", "pm_cs", "temp", "humidity"]].corr()
corr.style.background_gradient(cmap="coolwarm")

`pm_cs` and `pm_airnow` show high linear correlation of 0.89. 

In [ ]:
pd.plotting.scatter_matrix(full_data[["pm_airnow", "pm_cs", "temp", "humidity"]])
plt.show()

`pm_cs` and `pm_airnow` show high linear correlation. 